In [ ]:
# Testing Cell
from aviary.interface.methods_for_level2 import AviaryGroup
from aviary.utils.doctape import glue_variable

ag = AviaryGroup()
opts = list(ag.options)
for opt in opts:
    glue_variable(opt, md_code=True)

# Key SGM Capabilities

Shooting (or Forward in Time Integration) methods offer several benefits over collocation methods that some users might find useful:
* Trajectories are physical for every iteration of the optimization (including failed optimizations)
* Little to no initial guessing required for the trajectory
* Dynamically ordered events and phases

## Setting up ODEs

SGM expects all of the states (including time) as inputs to the ODE and the state rates as outputs.
If a particular state does not directly influence the EOM (such as distance in the GASP based climb_eom), it can be added using the built in helper functions:

In [ ]:
from aviary.interface.download_models import get_model
from aviary.mission.gasp_based.ode.rotation_ode import RotationODE
from aviary.mission.gasp_based.ode.time_integration_base_classes import SimuPyProblem
from aviary.mission.gasp_based.ode.two_dof_ode import TwoDOFODE
from aviary.subsystems.aerodynamics.aerodynamics_builder import CoreAerodynamicsBuilder
from aviary.subsystems.propulsion.propulsion_builder import CorePropulsionBuilder
from aviary.subsystems.propulsion.utils import build_engine_deck
from aviary.utils.process_input_decks import create_vehicle, update_GASP_options
from aviary.variable_info.enums import AnalysisScheme, LegacyCode
from aviary.variable_info.variables import Aircraft, Dynamic, Mission

aviary_inputs, initialization_guesses = create_vehicle(get_model('aircraft_for_bench_GwGm.csv'))
aviary_inputs = update_GASP_options(aviary_inputs)
aviary_inputs.set_val(
    Mission.Summary.CRUISE_MASS_FINAL, val=initialization_guesses['cruise_mass_final'], units='lbm'
)
aviary_inputs.set_val(
    Mission.Summary.GROSS_MASS, val=initialization_guesses['actual_takeoff_mass'], units='lbm'
)
aviary_inputs.set_val(Aircraft.Engine.NUM_ENGINES, val=[1], units='unitless')
default_mission_subsystems = [
    CoreAerodynamicsBuilder('core_aerodynamics', code_origin=LegacyCode.GASP),
    CorePropulsionBuilder('core_propulsion', engine_models=[build_engine_deck(aviary_inputs)]),
]
ode_args = {'aviary_options': aviary_inputs, 'core_subsystems': default_mission_subsystems}
simupy_args = {'aviary_options': aviary_inputs}

In [ ]:
from aviary.mission.gasp_based.ode.time_integration_base_classes import (
    add_SGM_required_inputs,
    add_SGM_required_outputs,
)


class ClimbODE(TwoDOFODE):
    # ## ... ## #
    def setup(self):
        # ## ... ## #
        analysis_scheme = self.options['analysis_scheme']
        # ## ... ## #
        if analysis_scheme is AnalysisScheme.SHOOTING:
            add_SGM_required_inputs(
                self,
                {
                    't_curr': {'units': 's'},
                    Dynamic.Mission.DISTANCE: {'units': 'ft'},
                    'alt_trigger': {'units': self.options['alt_trigger_units'], 'val': 10e3},
                    'speed_trigger': {'units': self.options['speed_trigger_units'], 'val': 100},
                },
            )

These functions allow the user to leave the EOMs unmodified for collocation vs shooting, and provide an easy way to set the units, default values, and any other keyword args for the OpenMDAO functions [add_input and add_output](https://openmdao.org/newdocs/versions/latest/features/core_features/working_with_components/continuous_variables.html) for any variables that only used by SGM.

## Setting up Phases

Each SGM phase should inherit from SimuPyProblem and requires an instantiated ODE. If no states are provided Aviary will attempt to determine the states in the current phase by finding the state rates (any output that ends in `'_rate'`). States and their rates are expected to have the same name (other than the addition of the `'_rate'` suffix for the state rate), if the state rate associated with a state doesn't follow this pattern, it can be specified through {glue:md}`alternate_state_rate_names`, a dictionary with state names as the keys and the desired state rate as the value.

In [ ]:
# Testing Cell
import inspect

from aviary.utils.doctape import check_args, check_value, glue_variable

rate_suffix = inspect.signature(SimuPyProblem).parameters['rate_suffix'].default
check_value(rate_suffix, '_rate')
check_args(SimuPyProblem, 'alternate_state_rate_names')
glue_variable('alternate_state_rate_names', md_code=True)

In [ ]:
class SGMRotation(SimuPyProblem):
    """
    This creates a subproblem for the rotation phase of the trajectory that will
    be solved using SGM.
    Rotation ends when the normal force on the runway reaches 0.
    """

    def __init__(
        self,
        phase_name='rotation',
        ode_args={},
        simupy_args={},
    ):
        super().__init__(
            RotationODE(analysis_scheme=AnalysisScheme.SHOOTING, **ode_args),
            problem_name=phase_name,
            outputs=['normal_force', 'alpha'],
            states=[
                Dynamic.Vehicle.MASS,
                Dynamic.Mission.DISTANCE,
                Dynamic.Mission.ALTITUDE,
                Dynamic.Mission.VELOCITY,
            ],
            # state_units=['lbm','nmi','ft'],
            alternate_state_rate_names={
                Dynamic.Vehicle.MASS: Dynamic.Vehicle.Propulsion.FUEL_FLOW_RATE_NEGATIVE_TOTAL
            },
            **simupy_args,
        )

        self.phase_name = phase_name
        self.add_trigger('normal_force', 0, units='lbf')

One of the main benefits of SGM is the ability to add arbitrarily ordered phases using triggers. Aviary uses an {glue:md}`event_trigger` class to store the information necessary for SGM phases. Instantiated {glue:md}`event_triggers` can be passed directly to the problem, or the helper function self.add_trigger can be used to generate the triggers. Triggers are generally used to check when the value of a state reaches a certain value, but can be used with any output from the ODE, such as `normal_force` in {glue:md}`SGMRotation`. Multiple triggers can be added to one phase, but the event will be triggered by whichever condition is met first.

In [ ]:
# Testing Cell
import inspect

from aviary.mission.gasp_based.ode.time_integration_base_classes import event_trigger
from aviary.mission.gasp_based.phases.time_integration_phases import SGMRotation
from aviary.mission.gasp_based.phases.time_integration_traj import FlexibleTraj
from aviary.utils.doctape import check_value, get_variable_name, glue_variable

# from aviary.interface.default_phase_info.two_dof_fiti import add_default_sgm_args

rotation_trigger: event_trigger = SGMRotation(ode_args=ode_args, simupy_args=simupy_args).triggers[
    0
]
check_value(rotation_trigger.state, 'normal_force')
glue_variable(get_variable_name(event_trigger), md_code=True)
glue_variable(get_variable_name(event_trigger) + 's', md_code=False)
glue_variable(get_variable_name(SGMRotation), md_code=False)
glue_variable(get_variable_name(FlexibleTraj), md_code=False)

signature = inspect.signature(SGMRotation.__init__)
# Print argument details
for param_name, param in signature.parameters.items():
    glue_variable(param_name, md_code=True)

glue_variable('two_dof_fiti', md_code=True)

## Setting up Trajectories

Aviary problems using the shooting method use {glue:md}`FlexibleTraj` to define their trajectories, instead of `dm.Trajectory()`. Similar to collocation problems, SGM will loop through the phases specified in the {glue:md}`phase_info` to build up the trajectory. When creating an SGM trajectory, the variables that will be used as inputs and outputs for states, triggers, and variables, including phase specific ones, are specified.

In [ ]:
from aviary.interface.default_phase_info.two_dof_fiti import add_default_sgm_args, phase_info
from aviary.mission.gasp_based.phases.time_integration_traj import FlexibleTraj

add_default_sgm_args(phase_info, ode_args)

full_traj = FlexibleTraj(
    Phases=phase_info,
    traj_final_state_output=[
        Dynamic.Vehicle.MASS,
        Dynamic.Mission.DISTANCE,
    ],
    traj_initial_state_input=[
        Dynamic.Vehicle.MASS,
        Dynamic.Mission.DISTANCE,
        Dynamic.Mission.ALTITUDE,
    ],
    traj_event_trigger_input=[
        # specify ODE, output_name, with units that SimuPyProblem expects
        # assume event function is of form ODE.output_name - value
        # third key is event_idx associated with input
        ('groundroll', Dynamic.Mission.VELOCITY, 0),
        ('climb3', Dynamic.Mission.ALTITUDE, 0),
        ('cruise', Dynamic.Vehicle.MASS, 0),
    ],
    traj_intermediate_state_output=[
        ('cruise', Dynamic.Mission.DISTANCE),
        ('cruise', Dynamic.Vehicle.MASS),
    ],
)

Because all phases require {glue:md}`ode_args` and {glue:md}`simupy_args` which are usually the same for all phases, {glue:md}`add_default_sgm_args` has been provided to add these to the phase info automatically.

### Setting up Phase Info

By default, SGM uses the {glue:md}`two_dof_fiti` (two degree of freedom, forward in time integration) phase_info, which contains the information required to build the default trajectory used by GASP. This phase info can be imported all at once or in a few pre-defined groups:
phase_info contains all the phases from {glue:md}`ascent_phases` (which is composed of {glue:md}`takeoff_phases` and {glue:md}`climb_phases`), {glue:md}`cruise_phase`, and {glue:md}`descent_phases`.

In [ ]:
from aviary.interface.default_phase_info.two_dof_fiti import (
    ascent_phases,
    climb_phases,
    cruise_phase,
    descent_phases,
    phase_info,
    takeoff_phases,
)

{glue:md}`phase_info_parameterization` can be used to update the values of certain variables, like speed_trigger or cruise_alt using values from the input deck.

## Descent Fuel Estimation

In the current formulation of the trajectory, the fuel and/or distance required for the descent are required a priori for the cruise trigger. This can be achieved by adding a submodel that contains just the descent trajectory before the main trajectory is created. The value that results from the aircraft flying this descent can then be connected to trigger values in the main trajectory.

The default descent that is flown using the `two_dof_fiti` phase_info is an idle descent.

In [ ]:
from aviary.mission.gasp_based.idle_descent_estimation import add_descent_estimation_as_submodel

In [ ]:
# Testing Cell
from aviary.utils.doctape import check_value

for phase_name, phase in descent_phases.items():
    check_value(phase['user_options'][Dynamic.Vehicle.Propulsion.THROTTLE], (0, 'unitless'))

In [ ]:
# Testing Cell
import aviary.api as av
from aviary.utils.doctape import get_function_names, get_variable_name

file_path = av.get_path('interface/default_phase_info/two_dof_fiti.py')
takeoff_function_names = get_function_names(file_path)
for func_name in takeoff_function_names:
    glue_variable(func_name, md_code=True)

glue_variable(get_variable_name(ascent_phases), md_code=False)
glue_variable(get_variable_name(takeoff_phases), md_code=False)
glue_variable(get_variable_name(climb_phases), md_code=False)
glue_variable(get_variable_name(cruise_phase), md_code=False)
glue_variable(get_variable_name(descent_phases), md_code=False)